In [ ]:
import os
import pathlib
import tarfile
import re

import numpy as np 
import pandas as pd

# 1. Install Tensorflow Object Detection 2

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("The TensorFlow version installed in the Notebook is TensorFlow {}".format(tf.__version__))
print("The TensorFlow version of Keras installed in the Notebook is Keras {}".format(keras.__version__))

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
!pip install tf_slim

In [ ]:
!pip install pycocotools
!pip install lvis
!pip install numba 

### Clone the Tensorflow models repository

In [ ]:
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research

# Compile protos.
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#Environment variables
os.environ['PYTHONPATH'] += ":/kaggle/working/models:/kaggle/working/models/research/slim/:/kaggle/working/models/research:"

import sys
sys.path.append("/kaggle/working/models")
sys.path.append("/kaggle/working/models/research")

In [ ]:
%%bash
cd models/research

# Test the installation.
python object_detection/builders/model_builder_tf2_test.py

# 2. Read the Data

In [ ]:
train_csv = pd.read_csv('../input/vinbigdata-original-image-dataset/vinbigdata/train.csv')

In [ ]:
train_csv.head()

In [ ]:
# Take the pictures with findings:
train_with_findings = train_csv[train_csv['class_name'] != 'No finding']

# 3. Transform Files to TF Records

In [ ]:
import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

debug = False

In [ ]:
def split(df, group):
    data = namedtuple('data', ['image_id', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [ ]:
def create_tf_example(group, path):
    
    # Read the file (jpg)
    with tf.gfile.GFile(os.path.join(path, '{}.jpg'.format(group.image_id)), 'rb') as fid:
        encoded_jpg = fid.read()
        
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    # TODO CHANGE
    width, height = image.size

    filename = group.image_id.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['x_min'] / width)
        xmaxs.append(row['x_max'] / width)
        ymins.append(row['y_min'] / height)
        ymaxs.append(row['y_max'] / height)
        classes_text.append(row['class_name'].encode('utf8'))
        classes.append(row['class_id'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    
    return tf_example

In [ ]:
grouped = split(train_with_findings, 'image_id')

In [ ]:
# TODO: Do the split right
# Split the train data into training and validation: 
# 70 % Train, 30 % validation

In [ ]:
grouped_train = grouped[0:3076]
grouped_val = grouped[3077:4393]

In [ ]:
path = os.path.join('../input/vinbigdata-original-image-dataset/vinbigdata/train')

In [ ]:
# Generating TRAINING TF Records
writer = tf.python_io.TFRecordWriter('/kaggle/working/train.record')

for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
writer.close()
output_path = os.path.join(os.getcwd(), '/kaggle/working/train.record')
print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
# Generating VALIDATION TF Records
writer = tf.python_io.TFRecordWriter('/kaggle/working/validation.record')

for group in grouped_train:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
writer.close()
output_path = os.path.join(os.getcwd(), '/kaggle/working/validation.record')
print('Successfully created the TFRecords: {}'.format(output_path))

# 4. Generate the Label Map File

In [ ]:
# Function taken from https://www.kaggle.com/sreevishnudamodaran/vbd-efficientdet-tf2-object-detection-api

from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from google.protobuf import text_format

def convert_classes(classes, start=1):
    
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))
    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

labels =  [
            "Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule_Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"
            ]

txt = convert_classes(labels)
print(txt)
with open('/kaggle/working/label_map.pbtxt', 'w') as f:
    f.write(txt)

# 5. Model Configuration

In [ ]:
train_record_fname = '/kaggle/working/train.record'
validation_record_fname = '/kaggle/working/validation.record'
label_map_pbtxt_fname = '/kaggle/working/label_map.pbtxt'

In [ ]:
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'faster-rcnn-resnet50_v1': {
        'model_name': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 4
    }
}

chosen_model = 'faster-rcnn-resnet50_v1'

num_steps = 3000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

### Download the pre-trained weights

In [ ]:
%mkdir /kaggle/working/deploy
%cd /kaggle/working/deploy

download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

### Download base training configuration file

In [ ]:
%cd /kaggle/working/deploy

download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

### Write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

In [ ]:
pipeline_fname = '/kaggle/working/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/kaggle/working/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)
num_classes

In [ ]:
%cd /kaggle/working/deploy/
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(validation_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
%cat /kaggle/working/deploy/pipeline_file.config

# 6. Train the Model

In [ ]:
%mkdir /kaggle/working/training/

In [ ]:
pipeline_file = '/kaggle/working/deploy/pipeline_file.config'
model_dir = '/kaggle/working/training/'

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

# 7. Evaluate the model

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir} \
    --eval_timeout=60

# 8. Run Inference on Test Images with Custom TensorFlow2 Object Detector

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import glob

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: the file path to the image

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    
    (im_width, im_height) = image.size
    rgb_img = image.convert('RGB')
    return np.array(rgb_img.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8), (im_width, im_height)

In [ ]:
%ls '/kaggle/working/training/'

In [ ]:
#recover our saved model
pipeline_file = '/kaggle/working/deploy/pipeline_file.config'
pipeline_config = pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = '/kaggle/working/training/ckpt-4'

configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/kaggle/working/training/ckpt-4'))

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

In [ ]:
detect_fn = get_model_detection_function(detection_model)

In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
image_np, size = load_image_into_numpy_array('/kaggle/input/vinbigdata-original-image-dataset/vinbigdata/test/002a34c58c5b758217ed1f584ccbcfe9.jpg')

input_tensor = tf.convert_to_tensor(
  np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'][0].numpy(),
    (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=.5,
    agnostic_mode=False
)

plt.figure(figsize=(8,8))
plt.imshow(image_np_with_detections)
plt.show()

In [ ]:
detection_scores = detections['detection_scores'][0].numpy()
detection_classes = np.array([category_index[x]['name'] for x in (detections['detection_classes'][0].numpy()  + label_id_offset).astype(int)])

In [ ]:
detection_threshold_class = detection_classes[detection_scores >= .5]
detection_threshold_class

In [ ]:
detection_threshold_score = detection_scores[detection_scores >= 0.5]
detection_threshold_score

# References

https://www.kaggle.com/awsaf49/vinbigdata-original-image-dataset

https://www.kaggle.com/sreevishnudamodaran/vbd-efficientdet-tf2-object-detection-api